In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46588
2,Huelva,Confirmados PDIA 14 días,467
3,Huelva,Tasa PDIA 14 días,"91,00298146812948"
4,Huelva,Confirmados PDIA 7 días,265
5,Huelva,Tasa PDIA 7 dias,"51,63980747120837"
6,Huelva,Total Confirmados,46791
7,Huelva,Curados,45397
8,Huelva,Fallecidos,421


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46588.0


/tmp/ipykernel_5714/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12818.0


/tmp/ipykernel_5714/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_5714/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_5714/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_5714/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1598 personas en los últimos 7 días 

Un positivo PCR cada 1027 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46588.0,265.0,467.0,513170.0,51.639807,91.002981,81.0
Condado-Campiña,14504.0,102.0,185.0,156231.0,65.287939,118.414399,44.0
Huelva-Costa,27536.0,140.0,247.0,289548.0,48.351223,85.305373,32.0
Huelva (capital),12818.0,90.0,140.0,143837.0,62.570827,97.332397,24.0
Moguer,2056.0,16.0,41.0,21867.0,73.169616,187.497142,11.0
Palos de la Frontera,1108.0,24.0,41.0,11742.0,204.394481,349.173906,11.0
Gibraleón,958.0,8.0,11.0,12737.0,62.809139,86.362566,6.0
Almonte,2352.0,14.0,29.0,24507.0,57.126535,118.333537,6.0
Sierra de Huelva-Andévalo Central,4121.0,22.0,34.0,67391.0,32.645309,50.451841,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,59.0,6.0,9.0,713.0,841.514727,1262.272090,1.0
Palos de la Frontera,1108.0,24.0,41.0,11742.0,204.394481,349.173906,11.0
Almonaster la Real,92.0,5.0,6.0,1788.0,279.642058,335.570470,0.0
Villanueva de las Cruces,21.0,1.0,1.0,387.0,258.397933,258.397933,0.0
Cartaya,2165.0,18.0,49.0,20083.0,89.628044,243.987452,3.0
San Bartolomé de la Torre,390.0,2.0,8.0,3761.0,53.177346,212.709386,1.0
Moguer,2056.0,16.0,41.0,21867.0,73.169616,187.497142,11.0
Bollullos Par del Condado,1320.0,22.0,23.0,14387.0,152.915827,159.866546,3.0
Minas de Riotinto,189.0,1.0,6.0,3812.0,26.232949,157.397692,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Minas de Riotinto,189.0,1.0,6.0,3812.0,26.232949,157.397692,0.0,0.166667
San Juan del Puerto,770.0,1.0,6.0,9411.0,10.625863,63.755180,0.0,0.166667
Aljaraque,1650.0,3.0,12.0,21474.0,13.970383,55.881531,1.0,0.250000
Trigueros,546.0,1.0,4.0,7862.0,12.719410,50.877639,0.0,0.250000
Ayamonte,2113.0,1.0,4.0,21104.0,4.738438,18.953753,0.0,0.250000
San Bartolomé de la Torre,390.0,2.0,8.0,3761.0,53.177346,212.709386,1.0,0.250000
Cartaya,2165.0,18.0,49.0,20083.0,89.628044,243.987452,3.0,0.367347
Moguer,2056.0,16.0,41.0,21867.0,73.169616,187.497142,11.0,0.390244
Almonte,2352.0,14.0,29.0,24507.0,57.126535,118.333537,6.0,0.482759
